In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
##load the dataset
data=pd.read_csv('lowdim_rna_prot_separated.csv',index_col=0)
label=np.genfromtxt('cell_type_no_onlyrna.csv',delimiter=',').astype('int64')


In [3]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable
from torch.optim import Adam
from torchvision.datasets.mnist import MNIST
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
%matplotlib inline

In [4]:
class ConvCaps2D(nn.Module):
    def __init__(self):
        super(ConvCaps2D, self).__init__()
        # The paper suggests having 32 8D capsules
        self.capsules = nn.ModuleList([nn.Conv2d(in_channels = 256, out_channels = 8, kernel_size=(1,4), stride=2)
                                       for _ in range(32)])
        
    def squash(self, tensor, dim=-1):
        norm = (tensor**2).sum(dim=dim, keepdim = True) # norm.size() is (None, 1152, 1)
        scale = norm / (1 + norm) # scale.size()  is (None, 1152, 1)  
        return scale*tensor / torch.sqrt(norm)
        
    def forward(self, x):
        outputs = [capsule(x).view(x.size(0), 8, -1) for capsule in self.capsules] # 32 list of (None, 1, 8, 36)
        outputs = torch.cat(outputs, dim = 2).permute(0, 2, 1)  # outputs.size() is (None, 1152, 8)
        return self.squash(outputs)

In [5]:
class Caps1D(nn.Module):
    def __init__(self):
        super(Caps1D, self).__init__()
        self.num_caps = 13
        self.num_iterations = 3
        self.W = nn.Parameter(torch.randn(13, 928, 8, 16))
        
    def softmax(self, x, dim = 1):
        transposed_input = x.transpose(dim, len(x.size()) - 1)
        softmaxed_output = F.softmax(transposed_input.contiguous().view(-1, transposed_input.size(-1)))
        return softmaxed_output.view(*transposed_input.size()).transpose(dim, len(x.size()) - 1)

    def squash(self, tensor, dim=-1):
        norm = (tensor**2).sum(dim=dim, keepdim = True) # norm.size() is (None, 1152, 1)
        scale = norm / (1 + norm)        
        return scale*tensor / torch.sqrt(norm)
   
    # Routing algorithm
    def forward(self, u):
        # u.size() is (None, 1152, 8)
        '''
        From documentation
        For example, if tensor1 is a j x 1 x n x m Tensor and tensor2 is a k x m x p Tensor, 
        out will be an j x k x n x p Tensor.
        
        We need j = None, 1, n = 1152, k = 10, m = 8, p = 16
        '''
        
        u_ji = torch.matmul(u[:, None, :, None, :], self.W) # u_ji.size() is (None, 10, 1152, 1, 16)
        
        b = Variable(torch.zeros(u_ji.size())) # b.size() is (None, 10, 1152, 1, 16)
        
        for i in range(self.num_iterations):
            c = self.softmax(b, dim=2)
            v = self.squash((c * u_ji).sum(dim=2, keepdim=True)) # v.size() is (None, 10, 1, 1, 16)

            if i != self.num_iterations - 1:
                delta_b = (u_ji * v).sum(dim=-1, keepdim=True)
                b = b + delta_b
        
        # Now we simply compute the length of the vectors and take the softmax to get probability.
        v = v.squeeze()
        classes = (v ** 2).sum(dim=-1) ** 0.5
        classes = F.softmax(classes) # This is not done in the paper, but I've done this to use CrossEntropyLoss.
        
        return classes
net = Caps1D()


In [6]:
class CapsNet(nn.Module):
    def __init__(self):
        super(CapsNet, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 256, kernel_size = (1,4), stride = 1)
        
        self.primaryCaps = ConvCaps2D()
        self.digitCaps = Caps1D()
        
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.primaryCaps(x)
        x = self.digitCaps(x)
        
        return x

net = CapsNet()

In [7]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())


In [8]:
def evaluate(model, X, Y, batch_size = 50):
    batch_size=len(X)//6
    results = []
    predicted = []
    for i in range(len(X)//batch_size):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = Variable(torch.from_numpy(X[s:e]))
        pred = model(inputs)
        
        predicted += list(np.argmax(pred.data.cpu().numpy(), axis = 1))

    Y=Y[0:len(predicted)]
    acc = sum(Y == predicted)*1.0/(len(Y))  
    return acc

In [11]:
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

In [12]:
batch_size=200
#net=CapsNet()
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.1)
d1=X_test.values.reshape(len(X_test),1, 64, order='F')
d1_test=np.expand_dims(d1.astype('float32'), 1)
l1_test=y_test

####performing 10-fold CV
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
cv = KFold(n_splits=10, random_state=42, shuffle=False)
trn_acc_cv=[]
vald_acc_cv=[]
trn_loss_cv=[]
vald_loss_cv=[]
test_acc=[]
k=1
for train_index, test_index in cv.split(X_train):
    #new_net=net
    print("fold",k)
    X_train1, X_val1, y_train1, y_val1 = X_train.iloc[train_index,:], X_train.iloc[test_index,:], y_train[train_index], y_train[test_index]
    #print(X_train[test_index].shape)
    d=X_train1.values.reshape(len(X_train1),1, 64, order='F')
    d1_train=np.expand_dims(d.astype('float32'), 1)
        #l1_train=y_train
    indices = np.random.permutation(len(d1_train))
    d1_train=d1_train[indices]
    l1_train=y_train1[indices]

    d1=X_val1.values.reshape(len(X_val1),1, 64, order='F')
    d1_vald=np.expand_dims(d1.astype('float32'), 1)
      # l1_test=y_test
    indices = np.random.permutation(len(d1_vald))
    d1_vald=d1_vald[indices]
    l1_vald=y_val1[indices]
   #liveloss = PlotLosses()
     

    loss_trn=[]
    loss_test=[]
    trn_acc = []
    vald_acc = []
    trn_loss =[]
    vald_loss=[]
    #net1=net
    #optimizer = optim.Adam(new_net.parameters())  

    for epoch in range(20):  # 20 epochs is enough for cite-seq
        #net1=net
        print("epoch",epoch)
        for phase in ['train', 'validation']:
            if phase == 'train':
                running_loss=0 
                for i in range(len(d1_train)//batch_size-1):    ##iteration
                    print(i,)
                    s = i*batch_size
                    e = i*batch_size+batch_size

                    inputs = torch.from_numpy(d1_train[s:e])
                    labels = torch.LongTensor(np.array(l1_train[s:e]))

                        # wrap them in Variable
                    inputs, labels = Variable(inputs), Variable(labels)

                        # zero the parameter gradients
                    optimizer.zero_grad()

                        # forward + backward + optimize
                    outputs = net(inputs)

                    loss = criterion(outputs, labels)
                    loss.backward()

                    optimizer.step()
                    running_loss += loss.data.item()
            #    print("Epoch, Loss - {}, {}".format(i, running_loss))
                    del inputs, labels
                    #print('\n')
                   # trn_loss.append(running_loss)
            else: 
                r=random.sample(range(1, len(d1_train)), 1000)
                trn_acc.append(evaluate(net, d1_train[r], l1_train[r], batch_size = 200)) 
                vald_acc.append(evaluate(net, d1_test, l1_test, batch_size=200)) 
                out_train=net(torch.from_numpy(d1_train[r]))
                out_test=net(torch.from_numpy(d1_vald))
                loss_trn = criterion(out_train, torch.LongTensor(np.array(l1_train[r])))
                loss_vald= criterion(out_test, torch.LongTensor(np.array(l1_vald)))
                trn_loss.append(loss_trn.data.item())
                vald_loss.append(loss_vald.data.item())
                print("train_acc",trn_acc)
                print("validation_acc",vald_acc)
                print("train_loss",trn_loss)
                print("validation_loss",vald_loss)
               # logs['log_loss_trn'] = loss_trn.append(loss_trn)
               # logs['log_loss_tst'] = loss_test.append(loss_test)
              #  logs['tr_accuracy'] = trn_acc[-1]
              #  logs['tst_accuracy'] = tst_acc[-1]
                #del net1
    print("kkkk")
    #del new_net   
    trn_acc_cv.append(trn_acc)
    vald_acc_cv.append(vald_acc)
    trn_loss_cv.append(trn_loss)
    vald_loss_cv.append(vald_loss)
    test_acc.append(evaluate(net, d1_test, l1_test, batch_size = 50))
    del trn_acc, trn_loss, vald_loss, vald_acc
    k=k+1
    net.apply(weights_init)
   # keras.backend.clear_session()
       # liveloss.update(logs)
        #liveloss.draw()
        #print("Epoch, Loss - {}, {}".format(epoch, running_loss))
        #print("Train - ", trn_acc[-1])
        #print("Test - ", tst_acc[-1])

fold 1
epoch 0
0


/export/scratch1/home/sumanta/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.
/export/scratch1/home/sumanta/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
train_acc [0.6506024096385542]
validation_acc [0.6323155216284987]
train_loss [2.5006136894226074]
validation_loss [2.4979190826416016]
epoch 1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
train_acc [0.6506024096385542, 0.6556224899598394]
validation_acc [0.6323155216284987, 0.6526717557251909]
train_loss [2.5006136894226074, 2.4920530319213867]
validation_loss [2.4979190826416016, 2.4900457859039307]
epoch 2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
train_acc [0.6506024096385542, 0.6556224899598394, 0.6767068273092369]
validation_acc [0.6323155216284987, 0.6526717557251909, 0.6844783715012722]
train_loss [2.5006136894226074, 2.4920530319213867, 2.488128423690796]
validation_loss [2.4979190826416016, 2.4900457859039307, 2.484513521194458]
epoch 3
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
train_acc [0

/export/scratch1/home/sumanta/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  This is separate from the ipykernel package so we can avoid doing imports until


fold 2
epoch 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
train_acc [0.6566265060240963]
validation_acc [0.6564885496183206]
train_loss [2.4942123889923096]
validation_loss [2.4923136234283447]
epoch 1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
train_acc [0.6566265060240963, 0.857429718875502]
validation_acc [0.6564885496183206, 0.8498727735368957]
train_loss [2.4942123889923096, 2.476024627685547]
validation_loss [2.4923136234283447, 2.4759840965270996]
epoch 2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
train_acc [0.6566265060240963, 0.857429718875502, 0.9106425702811245]
validation_acc [0.6564885496183206, 0.8498727735368957, 0.8994910941475827]
train_loss [2.4942123889923096, 2.476024627685547, 2.470383644104004]
validation_loss [2.4923136234283447, 2.4759840965270996, 2.4697861671447754]
epoch 3
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
2

In [16]:
##save the results
from numpy import savetxt
savetxt('caps_citeseq_trnacc_cv.csv', trn_acc_cv, delimiter=',')
savetxt('caps_citeseq_valdacc_cv.csv', vald_acc_cv, delimiter=',')
savetxt('caps_citeseq_tstacc_cv.csv', test_acc, delimiter=',')

In [17]:
trn_acc_cv

[[0.6506024096385542,
  0.6556224899598394,
  0.6767068273092369,
  0.7038152610441767,
  0.7640562248995983,
  0.7951807228915663,
  0.8413654618473896,
  0.8785140562248996,
  0.8875502008032129,
  0.9056224899598394,
  0.9307228915662651,
  0.9216867469879518,
  0.927710843373494,
  0.9337349397590361,
  0.9327309236947792,
  0.9497991967871486,
  0.9447791164658634,
  0.9387550200803213,
  0.9397590361445783,
  0.9457831325301205],
 [0.6566265060240963,
  0.857429718875502,
  0.9106425702811245,
  0.9337349397590361,
  0.9397590361445783,
  0.929718875502008,
  0.9367469879518072,
  0.9437751004016064,
  0.9347389558232931,
  0.9487951807228916,
  0.9568273092369478,
  0.9357429718875502,
  0.9407630522088354,
  0.9467871485943775,
  0.9437751004016064,
  0.9457831325301205,
  0.9417670682730924,
  0.9508032128514057,
  0.9497991967871486,
  0.9548192771084337],
 [0.6335341365461847,
  0.8493975903614458,
  0.9196787148594378,
  0.9136546184738956,
  0.928714859437751,
  0.93473895